# Revisão com algumas funções importantes para a PI de RobComp

In [1]:
import cv2
import numpy as np

# OpenCV

## Transformando Imagens

In [ ]:
# Transformar imagem em RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Transformar imagem em HSV
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Transforma em GRAY
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## Criando Máscaras

In [ ]:
hsv_lower = np.array([0, 0, 0])
hsv_upper = np.array([255, 255, 255])

mask = cv2.inRange(img_hsv, hsv_lower, hsv_upper)

# Juntando Máscaras
mask_junta = cv2.bitwise_or(mask1, mask2)

# Se for necessário, podemos suavisar as bordas da imagem com a seguinte função:
kernel=np.ones((4,4), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel) 

## Filtrando Cores

In [ ]:
mascara = np.zeros_like(gray)
mascara[gray > 20] = 255
# Define como branco todos os pixels maiores que 20 na escala cinza

filtro_vermelho1 = np.zeros_like(gray)
filtro_vermelho1[bgr[:,:,2] > 200] = 255
# Define como branco todos os pixels maiores que 200 dentro do intervalo do vermelho

## Contornos

### Achar Contornos
- `mask` é a imagem com a máscara binária
- `cv2.RETR_CCOMP` indica que queremos organizar os contornos em componentes conexos e buracos dentro deles
- `cv2.CHAIN_APPROX_NONE` indica que queremos armazenar todos os pontos do contorno
- `contours` é uma lista de contornos, contendo os pontos a ele pertencententes (x, y)
- `hierarchy` é uma lista indicando a organização dos contornos em termos dos componentes e de seus buracos

A função retorna três valores: os contornos propriamente ditos, a hierarquia dos contornos e o método de aproximação de contornos utilizado.

- Contornos: é uma lista Python de todos os contornos encontrados na imagem. Cada contorno é representado por uma matriz NumPy de pontos (x,y) que o formam.
- Hierarquia: é uma matriz NumPy que define a relação de cada contorno com os outros contornos encontrados. Ela pode ser usada para identificar contornos internos ou externos, por exemplo.
- Método de aproximação de contornos: é um parâmetro opcional que define o método de aproximação de contornos utilizado. O valor padrão é cv2.CHAIN_APPROX_SIMPLE.


In [ ]:
# É possível encontrar componente conexos em imagens tons de cinza através da função cv2.findContours(). 
# Ela considera pixels de valor maior do que 0 como pixels de interesse
contornos, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP/cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [ ]:
# Lista das coordenadas x e y dos contornos
x_list = contornos[0][:,:,0]
y_list = contornos[0][:,:,1]

### Achar os 4 maiores contornos

In [ ]:
contornos = sorted(contornos, key=cv2.contourArea)[-4]

### Printar Contornos
- `imagem` é a imagem colorida ou tons de cinza a receber o contorno
- `contours` é a lista de contornos obtida com `cv2.findContours()`
- `indice` é o índice do contorno dentro da lista a ser desenhado; se indice < 0 desenha todos os contornos
- `cor` é a cor do pixel a ser usada para desenhar o contorno   

In [ ]:
# Mostrar contornos na imagem
cv2.drawContours(img, contornos, indice, cor)

# Exemplo:
cv2.drawContours(img, contornos, -1, [0, 0, 255], 3) # 3 representa a espessura do contorno

## Área, Centro de Massa e Texto na Tela

In [ ]:
# Área dos contornos:
area = cv2.contourArea(contours[indice]) # Recebe apenas um contorno por vez

# Retorna a área em pixels

In [ ]:
# Centro de Massa
def center_of_contour(contorno): # Recebe apenas um contorno por vez
    """ Retorna uma tupla (cx, cy) que desenha o centro do contorno"""
    M = cv2.moments(contorno)
    # Usando a expressão do centróide definida em: https://en.wikipedia.org/wiki/Image_moment
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return (int(cX), int(cY))
    
def crosshair(img, point, size, color):
    """ Desenha um crosshair centrado no point.
        point deve ser uma tupla (x,y)
        color é uma tupla R,G,B uint8
    """
    x,y = point
    cv2.line(img,(x - size,y),(x + size,y),color,5)
    cv2.line(img,(x,y - size),(x, y + size),color,5)
    
font = cv2.FONT_HERSHEY_SIMPLEX

def texto(img, a, p):
    """Escreve na img RGB dada a string a na posição definida pela tupla p"""
    cv2.putText(img, str(a), p, cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)

In [ ]:
# Exemplo:
# Vamos percorrer a lista de contornos e aplicar as funções de centro de massa definidas acima

for c in contornos:
    a = cv2.contourArea(c) # área
    p = center_of_contour(c) # centro de massa
    crosshair(img, p, 20, (128,128,0)) # desenha o crosshair
    texto(img, np.round(a,2),p) # escreve a área

## Bordas
- `mask`: máscara com valores em preto e branco.
- `threshold1`: O valor mínimo do limiar para a detecção de bordas. Qualquer borda com intensidade abaixo desse valor será descartada.
- `threshold2`: O valor máximo do limiar para a detecção de bordas. Qualquer borda com intensidade acima desse valor será considerada uma borda verdadeira.

In [ ]:
# Bordas utilizando o Canny:
bordas = cv2.Canny(mask, 100, 200) # vamos padronizar o uso de 100 e 200 como limiares mas é possível alterar

# Retorna a imagem com as bordas em preto e fundo em branco

## Linhas

- `image`: A imagem de entrada, que pode ser uma matriz NumPy ou um objeto cv2.Mat.
- `rho`: A resolução do parâmetro rho da transformada de Hough em pixels. Geralmente é 1.
- `theta`: A resolução do parâmetro theta da transformada de Hough em radianos. Geralmente é np.pi/180, que é aproximadamente 1 grau.
- `threshold`: O valor do limiar usado para a detecção de linhas. A função irá retornar apenas as linhas que têm um número de votos maior do que o valor de limiar.
- `minLineLength`: O comprimento mínimo da linha a ser detectada. Qualquer linha com comprimento abaixo desse valor será descartada.
- `maxLineGap`: A distância máxima permitida entre segmentos de linha para serem conectados em uma única linha. Qualquer lacuna entre segmentos de linha maior do que esse valor será considerada como duas linhas separadas.
- A função retorna uma matriz NumPy que representa as linhas detectadas na imagem. Cada linha é representada por um vetor de quatro elementos [x1 (inicial), y1 (inicial), x2 (final), y2 (final)] que especifica as coordenadas dos dois pontos que definem a linha.

In [ ]:
linhas = cv2.HoughLinesP(bordas, 10, np.pi/180.0, threshold=150, minLineLength=50, maxLineGap=70) # Apenas um exemplo, pode ser alterado

## Círculos
- `image`: Imagem de 8 bits onde as circunferências serão procuradas
- `method`: método usado para encontrar os possíveis centros de cada circunferência. Aqui vamos usar `cv2.HOUGH_GRADIENT`.
- `dp`: resolução usada na procura pelos centros das circunferências
- `minDist`: menor distância permitida entre os centros das circunferências encontradas
- `param1`: limiar empregado na detecção dos pontos de borda
- `param2`: limiar de detecção da circunferência
- `minRadius`: menor raio da circunferência a ser encontradas
- `maxRadius`: maior raio da circunferência a ser encontradas
- A função retorna uma matriz NumPy que representa os círculos detectados na imagem. Cada círculo é representado por um vetor de três elementos [x, y, r] que especifica as coordenadas do centro do círculo e seu raio.

In [ ]:
# Primeiro vamos transformar a imagem em tons de cinza
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Vamos achar os círculos (valores arbitrários)
circles = cv2.HoughCircles(grey,
                          cv2.HOUGH_GRADIENT,
                          dp=1,
                          minDist=20,
                          param1=50, 
                          param2=30,
                          minRadius=17,
                          maxRadius=50)
circles = np.uint16(np.around(circles))

## Printar Círculos
- `imagem`: a imagem na qual o círculo será desenhado.
- `centro`: as coordenadas do centro do círculo na imagem, no formato (x, y).
- `raio`: o raio do círculo a ser desenhado, em pixels.
- `cor`: a cor do círculo, no formato (B, G, R) ou escala de cinza.
- `espessura`: a espessura da linha que desenha o círculo. Se for negativo, o círculo será preenchido.

In [ ]:
cv2.circle(imagem, centro, raio, cor, espessura)

# MobileNet

## Classes e Cores

In [ ]:
# Apenas um exemplo, pode ser que mude de acordo com a questão
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

## Carregar MobileNet

In [1]:
def load_mobilenet():
    """Não mude ou renomeie esta função
        Carrega o modelo e os parametros da MobileNet. 
        Retorna a rede carregada.
    """
    # Load the model.
    net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt.txt', 'MobileNetSSD_deploy.caffemodel') #Parâmetros depende do arquivo
    return net

## Função Detect
- Não tente entender por completo kkkkk
- Recebe uma imagem/frame e a net carregada anteriormente

In [ ]:
def detect(frame, net):
    image = frame.copy()
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # pass the blob through the network and obtain the detections and
    # predictions
    print("[INFO] computing object detections...")
    net.setInput(blob)
    detections = net.forward()

    results = []

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence

        # Pode-se mudar o nível de confiança, mas alterará a precisão do resultado final, faça testes e decida o
        # melhor valor.
        if confidence > 0.5: 
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            print("[INFO] {}".format(label))
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

            results.append((CLASSES[idx], confidence*100, (startX, startY),(endX, endY) ))
        
    return results

## Chamando a função e printando resultados
- Recebe um frame/imagem e a função load_mobilenet()
- `res` Retorna: o nome do objeto detectado, a acurácia de ser esse objeto (%) e duas tuplas (x1, y1), (x2, y2). Essas tuplas representam coordendas da diagonal do retângulo. Ex: [('dog', 94.97159123420715, (183, 383), (336, 536))]  

In [ ]:
resultados = detect(frame, load_mobilenet())
infos = resultados[0] # Ex: ('dog', 94.97159123420715, (183, 383), (336, 536))

In [ ]:
# Exemplo
for resultado in resultados:
    x1 = resultado[2][0]
    y1 = resultado[2][1]
    x2 = resultado[3][0]
    y2 = resultado[3][1]

## Próximos Passos
- Agora depende de cada problema, have fun =)